#  RAG and Python function Deployment

In this notebook, we create and deploy a python function that takes a user defined question as input and generates an answer using the RAG process. The function does the following:
- Connects to Elasticsearch or Milvus or Watsonx.data Milvus Vector Database and retrieves the top `N` results (over a certain threshold) matching the question from the vector index.
- Includes the question, vector search results into an optimized prompt template for the LLM
- Runs LLM inference on watsonx.ai to generate an answer
- Checks for hallucinations using word overlap or sentence transformer algorithms and threshold values.
- Updates the feedback provided by the user based on the LLM result in log records. 
- Connects to Elasticsearch or Milvus or Watsonx.data Milvus Vector Database and retrieves the top `k` expert profile recommendation (over a certain threshold) matching the question from the vector index.
- Updates the recommended expert profile in log records. 
- Function that retrieves answers by auto-completing the question based on a given question prefix.


**Note**: It is recommended to run this notebook in a Python environment on CPD software with a GPU-enabled or high vCPU and RAM hardware configuration, as generating embeddings may require significant memory.
## Contents

This notebook contains the following parts:

- [Pre-Requisite Libraries and Dependencies](#Setup)
- [Import Parameter Set, Credentials and Helper function script](#parameterimport)
- [Initialise Deployment Space](#DeploymentSpace)
- [Promote Assets to Deployment Space](#promote)
- [Create Scoring Function](#ScoringFunction)
- [Deploy Scoring Function in Deployment Space](#DeployScoringFunction)
- [Test the scoring function](#scoring)
- [Test the expert recommendation function](#expert-recommendation)
- [Update the feedback log](#feedback-logging)
- [Test Auto complete](#test-auto-complete)
- [Update parameter set in the project & deployment space](#updateParameters)



<a id="setup"></a>
### Pre-Requisite Libraries and Dependencies
Download and import mandatory libraries and dependencies. 

Note : Some of the versions of the libraries may throw warnings after installation. These library versions are crucial for successful execution of the accelerator. Please ignore the warning/error and proceed with your execution. 

In [ ]:
!pip install elasticsearch==8.17.2 | tail -n 1
!pip install ibm_watsonx_ai==1.2.10 | tail -n 1
!pip install langchain_milvus==0.1.8 | tail -n 1

Restart the kernel after performing the pip install if the below cell fails to import all the libraries.

In [ ]:
import os
import json
import string
import time
import uuid
import re
from ibm_watsonx_ai import APIClient,Credentials
from ibm_watsonx_ai import __version__

import warnings
warnings.filterwarnings("ignore")

from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, PromptTemplateFormats
from pymilvus import(connections,FieldSchema,DataType,Collection,CollectionSchema,utility)

In [ ]:
project_id=os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()

<a id="parameterimport"></a>
### Import Parameter Sets, credentials and Helper functions script.

The below cells imports the parameter set, credentials for watsonx.ai and helper function script. 

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]

parameters=rag_helper_functions.get_parameter_sets(wslib, parameter_sets)

### Set Watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space. 

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']
space_uid = parameters['watsonx_ai_space_id']

if environment == "cloud":
    WML_SERVICE_URL = f"https://{hostname.split('.')[1]}.ml.cloud.ibm.com" if hostname.split('.')[0].endswith("api") and hostname.split('.')[2] == "dataplatform" else "https://us-south.ml.cloud.ibm.com"
    wml_credentials = Credentials(api_key=parameters['watsonx_ai_api_key'], url=WML_SERVICE_URL)
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials=Credentials(token=os.environ['USER_ACCESS_TOKEN'],url=hostname,instance_id='openshift')

In [ ]:
client = APIClient(wml_credentials)
client.set.default_project(project_id=project_id)

### Import Prompt Template

Imports the prompt template based on the parameter set by the user. 

In [ ]:

prompt_mgr = PromptTemplateManager(
                credentials=wml_credentials,
                project_id=project_id
                )


df_prompt = prompt_mgr.list()
prompt_template_id=df_prompt.loc[df_prompt['NAME'] == parameters['llm_prompt_template_file'], 'ID'].values[0]
prompt_model = prompt_mgr.load_prompt(prompt_template_id)
prompt_model_id=prompt_model.model_id

print("Currently using the ", parameters['llm_prompt_template_file'], "prompt template.")


### Create Connections from the project

Retrieves the connection information from the connection asset you created if the asset exists. \
Connects to the vector database depending on whether its an elasticsearch or milvus or watsonx.data milvus connection and initilizes a client respectively.

In [ ]:
connection_name=parameters["connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == connection_name), None)):
    print(connection_name, "Connection found in the project")
    db_connection = wslib.get_connection(connection_name)
    
    connection_datatypesource_id=client.connections.get_details(db_connection['.']['asset_id'])['entity']['datasource_type']
    connection_type = client.connections.get_datasource_type_details_by_id(connection_datatypesource_id)['entity']['name']
    
    print("Successfully retrieved the connection details")
    print("Connection type is identified as:",connection_type)

    if connection_type=="elasticsearch":
        es_client=rag_helper_functions.create_and_check_elastic_client(db_connection, parameters['elastic_search_model_id'])
    elif connection_type=="milvus" or connection_type=="milvuswxd":
        milvus_credentials = rag_helper_functions.connect_to_milvus_database(db_connection, parameters)

    project_connection_id = db_connection['.']['asset_id']
    

else:
    db_connection=""
    raise ValueError(f"No connection named {connection_name} found in the project.") 

The notebook, by default, will look for a log connection asset in the **RAG_parameter_set** (by default named `milvus_connect` or `elasticsearch_connect`). You can set this up by following the instructions in the project readme. This code checks if a specified connection exists in the project. If found, it retrieves the connection details and identifies the connection type. This can either be Elasticsearch or Milvus. \
Depending on the connection type, it establishes a connection to the appropriate database. If the connection is not found, it raises an error indicating the absence of the specified connection in the project.

In [ ]:
log_connection_name=parameters["log_connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == log_connection_name), None)):
    print(log_connection_name, "log Connection found in the project")
    log_db_connection = wslib.get_connection(log_connection_name)
    
    connection_datatypesource_id=client.connections.get_details(log_db_connection['.']['asset_id'])['entity']['datasource_type']
    log_connection_type = client.connections.get_datasource_type_details_by_id(connection_datatypesource_id)['entity']['name']
    
    print("Successfully retrieved the connection details")
    print("Connection type is identified as:",log_connection_type)

    if log_connection_type=="elasticsearch":
        log_client=rag_helper_functions.create_and_check_elastic_client(log_db_connection, parameters['elastic_search_model_id'])
    elif log_connection_type=="milvus" or connection_type=="milvuswxd":
        log_client = rag_helper_functions.connect_to_milvus_database(log_db_connection, parameters)
    project_log_connection_id = log_db_connection['.']['asset_id']

    
else:
    db_connection=""
    raise ValueError(f"No connection named {log_connection_name} found in the project.") 

 
<a id="DeploymentSpace"></a>
### Get the Deployment Space details and set default space.

In the next steps we save and deploy the pipeline. The pipeline can be saved and deployed in the same way we save and deploy models.

Before we deploy a function we must create a deployment space. Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their models. We can save models, functions and data assets in this space.

**Creating a Deployment Space** <br>
Before we save the function we must create a deployment space. Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their functions or models. We can save models, functions and data assets in this space.
If you do not have space already created, you can use Deployment Spaces Dashboard to create one. Follow the steps : 
* Navigate to Deployments
* Click New Deployment Space
* Enter Deployment space name, for eg : **'RAG Deployment Space'**

**In case of watsonx as a service**
* Follow the steps to create the space same as above. 
* Select Cloud Object Storage
* Select Watson Machine Learning instance and press Create
* `Under Manage project` > `Services & Integrations` ensure that your WML service that you provisioned is associated there. 


The steps involved in saving and deploying the pipeline are detailed in the following cells. We will use the `ibm-watson-machine-learning` package to complete these steps.

**Setting space id in parameters set**
* In your Deployment Space, copy the space ID by going to **Manage >** copy the Space GUID
* In your Project, open the `RAG_parameter_set` found in **Data Asset > Configuration**.
* Edit the parameter name `watsonx_ai_space_id` and paste the guid here.

In [ ]:
space = client.set.default_space(space_uid)


<a id="promote"></a>
### Promote Necessary Assets to the Deployment Space.

The following assets are promoted to the deployment space to be used in the deployed function
* Elasticsearch / Milvus Connection
* Parameter sets
* LLM Prompt template 
* RAG Helper functions python script

`promote_assets` method in rag_helper function that promotes a specified asset (data assets, connections or parameter) to a deployment space. It first checks if an asset with the given name already exists in the deployment space and, if the parameter `reuse_existing_space_assets` is set to 'True', reuses the existing asset's ID. If the asset does not exist or reuse is not enabled, the function promotes the asset from the project to the deployment space, creating a new asset ID. Finally, it returns the deployment space asset ID.

In [ ]:
connection_id  = rag_helper_functions.promote_assets(client, "connections", connection_name, parameters, project_connection_id, project_id, space_uid)

if log_connection_name != connection_name:
    log_connection_id=rag_helper_functions.promote_assets(client, "connections", log_connection_name, parameters, project_log_connection_id, project_id, space_uid)
else:
    log_connection_id = connection_id
        
    

promote_files_to_space = [
    "rag_helper_functions.py",
    parameters['elastic_search_template_file']
]

space_asset_dict={}
for file in wslib.assets.list_assets('data_asset'):
    if file['name'] in promote_files_to_space:
        space_asset_dict[file['name']]=rag_helper_functions.promote_assets(client, 'data_assets', file['name'], parameters, file['asset_id'], project_id, space_uid)


for parms in wslib.assets.list_assets('parameter_set'):
    if parms['name'] in parameter_sets:
        space_asset_dict[parms['name']] = rag_helper_functions.promote_assets(client, 'parameter_sets', parms['name'], parameters, parms['asset_id'], project_id, space_uid)
        

space_asset_dict

Promote the assets and the prompt template to the space to be used by the deployed function.

## Deploy prompt template

This code automates the deployment of a "QnA with RAG prompt template" based on whether a deployment ID is provided. If `parameters['prompt_deployment_id']` is empty, the code promotes the prompt template (`prompt_template_id`) to a deployment space (`space_uid`), defines deployment metadata (including the name, configuration, and base model ID), and deploys it, saving the deployment ID. If a deployment ID is already specified, it skips promotion and deployment. This ensures efficient handling of new or existing deployments.

In [ ]:
if not parameters['prompt_deployment_id']:
    print("Promoting the prompt template to deployment space")
    space_prompt_template_id=client.spaces.promote(prompt_template_id, project_id, space_uid)
    print("Deploying the prompt template..")
    meta_props = {
        client.deployments.ConfigurationMetaNames.NAME: "QnA with RAG prompt template",
        client.deployments.ConfigurationMetaNames.ONLINE: {},
        client.deployments.ConfigurationMetaNames.BASE_MODEL_ID: prompt_model_id}
    
    prompt_deployment_details = client.deployments.create(artifact_uid=space_prompt_template_id, meta_props=meta_props)
    prompt_deployment_id=prompt_deployment_details.get("metadata").get("id")
else:
    prompt_deployment_id=""
    print("The notebook will use the prompt template specified in the parameter set")
    

Asset details of all data assets promoted to the space to be used by the deployed function

In [ ]:
ai_params = {'space_id': space_uid, 'space_asset_dict': space_asset_dict, 'environment': environment, 
             'connection_name': connection_name, 'connection_id': connection_id,
             "connection_type":connection_type,"project_id":parameters.get('watsonx_ai_project_id') or project_id,
             'log_connection_id': log_connection_id, 'log_connection_type': log_connection_type,'prompt_deployment_id':parameters['prompt_deployment_id'] if parameters['prompt_deployment_id'] else prompt_deployment_id,
             'log_pii_removal': ('log_pii_removal' in parameters and parameters['log_pii_removal'].lower() == 'true'),'wml_credentials':wml_credentials.to_dict() }


### Create Software Specifications
Below code snippet performs the following actions:

It defines various configuration constants for setting up a Python runtime environment, including the base software specification, custom software specification name, package extension name, and a configuration file path. It creates a configuration content string in YAML format that specifies the dependencies needed. The configuration content is then written to a file named `config.yaml`. Metadata properties are prepared for storing the package extension using the configuration file, and these properties are saved to the IBM Watson Machine Learning client. The unique identifier (UID) for the stored package extension is retrieved. Metadata properties are also prepared for a new software specification that uses the base specification, and this is stored to the Watson Machine Learning client. The unique identifier (UID) for the newly created software specification is retrieved. Finally, the package extension is associated with the new software specification by adding the package extension UID to it.

The respective software specifications uid as a result is provided as a parameter during the deployment of function, so that pipeline function will have the dependent libraries for execution on the Deployment Space at runtime.

In [ ]:
BASE_SW_SPEC_NAME = "runtime-24.1-py3.11"
sw_spec_name = "rag_qna_sw_spec_"+str(uuid.uuid1()).split('-')[0]
pkg_ext_name = "rag_qna-py3.11"
CONFIG_PATH = "config.yaml"
CONFIG_TYPE = "conda_yml"
CONFIG_CONTENT = f"""
        name: python310
        channels:
          - empty
        dependencies:
          - pip:
            - langchain
            - setuptools
            - langchain_milvus==0.1.8
            - langchain-core
            - langchain_ibm
            - ibm-watsonx-ai=={__version__}
            - elasticsearch
            - langchain-community
            - langchain_chroma
            - langchain_elasticsearch==0.3.2
        prefix: /opt/anaconda3/envs/python310
"""
with open(CONFIG_PATH, 'w', encoding='utf-8') as f:
    f.write(CONFIG_CONTENT)
pkg_extn_meta_props = {
    client.package_extensions.ConfigurationMetaNames.NAME: pkg_ext_name,
    client.package_extensions.ConfigurationMetaNames.TYPE: CONFIG_TYPE
}

pkg_extn_details = client.package_extensions.store(meta_props=pkg_extn_meta_props, file_path=CONFIG_PATH)
pkg_extn_uid = client.package_extensions.get_id(pkg_extn_details)

sw_spec_meta_props = {
    client.software_specifications.ConfigurationMetaNames.NAME: sw_spec_name,
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        'guid': client.software_specifications.get_id_by_name(BASE_SW_SPEC_NAME)
    }
}

try:
    sw_spec_details = client.software_specifications.store(meta_props=sw_spec_meta_props)
    sw_spec_id = client.software_specifications.get_id(sw_spec_details)

    client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_uid)
except Exception as e:
    print(f"An error occurred: {e}")
    print("\nExisting software_specification will be used")
    sw_spec_id=client.software_specifications.get_id_by_name(sw_spec_name)
    client.software_specifications.add_package_extension(sw_spec_id, pkg_extn_uid)
    
import os
os.remove(CONFIG_PATH)

<a id="ScoringFunction"></a>

### Python scoring pipeline function

Functions can be deployed in Watson Machine Learning in the same way models can be deployed. The python client or REST API can be used to send data to the deployed function. Using the deployed function allows us to perform the list of tasks below and pass it to the pipeline for getting responses from the LLM, analysing and logging the response all within the deployed function.
The function handles the following tasks:

1. Initializing the WML client.
2. Downloading assets promoted to the deployment space.
3. Calling the Python helper functions script.
4. Initializing the Vector Database client and creating a vector store for Elasticsearch or Milvus or Watsonx.data Milvus.
5. Initializing LLM with parameters.
6. Creating an LLM chain using the LLM, vector store, and prompt template.
7. Retrieving a response from the LLM using LLM Chain and checking for hallucinations.
8. Writing response data to a log record in Elasticsearch or Milvus or Watsonx.data Milvus and creating an index/collection for logs if it doesn't exist. For this purpose it follows below steps:
   - Make connection to vector store specified in log connection type, i.e, either milvus or elastic search. 
   - Depending on the connection type
     - If connection type is 'Milvus' or 'Milvuswxd', it uses pymilvus to setup connection. An embedding model specified in the parameters is initialized for generating embeddings, and index parameters are defined for indexing with specific metrics and settings. If collection is not present, then collection with specific field/document is created to store log records. 
     - If connection type is 'Elasticsearch', it uses langchain_elasticsearch to setup connection. If index is not present, then the index is created to store log records.
   - Returning the result to the user.
   - Providing the document ID of the log record.
   - Updating the log record with the user's feedback (through a separate function call).
9. Retrive the top recommended experts for the question asked by user. Follow the below steps:
   - Make connection to vector store for expert profile as per the document connection type to either milvus or elastic search
   - If the relevant expert detail is found in the log record, retrieve the expert details and return to user.
   - If expert detail is not present, then retrieve the most relevant expert(s) from the expert profile index and return to user.
10. Test auto complete function by passing the question_prefix & limit values
    - Establish a connection either via milvus or elastic search to access the log index. 
    - In case there are existing records in log index that match a given question prefix, those matching questions are retrieved from the database based on the prefixed input value.
    - By default only top 5 questions are retrieved from the database based on the prefix. The parameter needs to be updated accordingly for this.

**If the custom query template for Elasticsearch has a different structure/format from the one as expected in the RAG pipeline, then you need to provide a mapper function which can format the response as required in the pipeline.**
Follow below steps to add custom document mapper for Elasticsearch query template:
1. Create a document mapper function inside the scoring pipeline function. Below is the sample document mapper function for nested query template.
   ```
       # Document Mapper for nested query template
       def document_mapper(hit):
           from langchain_core.documents import Document
           # 'passages' is the nested field
           if 'passages' in hit["_source"]:
                passages = hit["_source"]['passages']
                    
                return [Document(
                    vector=passage['sparse'],
                    page_content=passage['text'],
                    metadata={'_source': {'metadata': {"page_number": '', "source": hit['_source']['url_path'], "title": passage['title'], "document_url": passage['url']}}, "_score": hit['_score']},
                ) for passage in passages ]
   ```
     <br>
2. Comment out the `content_field` and Uncomment the `document_mapper` field in `ElasticsearchRetriever` and provide the added mapper function name as the value.

In [ ]:
def rag_scoring_pipeline_function(params=ai_params):

     # Note : To run the function locally you need to also uncomment all the subprocess calls below to install the dependent libraries. 
     # Similarly, ensure that they are commented out before the function is deployed in the deployment space to avoid library conflicts. (In very rare cases a kernel restart may be required.)"""
    
    #import subprocess
    #subprocess.check_output("pip install --upgrade elasticsearch --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain-community --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain-core --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain-elasticsearch==0.3.2 --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install ibm-watsonx-ai==1.2.10 --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain_ibm --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain_milvus==0.1.8 --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install langchain_chroma --user", stderr=subprocess.STDOUT, shell=True)
    #subprocess.check_output("pip install pydantic --user", stderr=subprocess.STDOUT, shell=True)


    from langchain_ibm import WatsonxLLM
    import requests
    import os
    import json
    import warnings
    import time
    import hashlib
    from datetime import datetime
    import re
    import math
    from functools import reduce
    import elasticsearch
    from elasticsearch import Elasticsearch
    from ibm_watsonx_ai import APIClient
    from ibm_watsonx_ai.foundation_models import Embeddings
    from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
    from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
    from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
    from ibm_watsonx_ai.foundation_models import ModelInference
    from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
    from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, PromptTemplateFormats, DecodingMethods
    from ibm_watsonx_ai import Credentials
    from ibm_watson_studio_lib import access_project_or_space
    import string
    from langchain.chains import LLMChain
    from langchain.schema.runnable import RunnableMap
    from collections import Counter
    import numpy as np
    from sklearn.metrics.pairwise import cosine_similarity
    from pymilvus import(IndexType,Status,connections,FieldSchema,DataType,Collection,CollectionSchema,utility)
    from langchain_milvus import Milvus, BM25BuiltInFunction    
    import copy
    from ibm_watsonx_ai.foundation_models.utils import HAPDetectionWarning
    import warnings    
    warnings.filterwarnings("always", category=HAPDetectionWarning)

    # Conditional import, since watson_nlp might not be available in runtime environment
    watson_nlp_available = False
    if params['log_pii_removal']:
        try:
            import watson_nlp
            watson_nlp_available = True
        except ImportError:
            pass

    # from langchain.globals import set_debug
    # set_debug(True)

    # AI Params passsed to function
    wml_credentials= params['wml_credentials']

    hostname = wml_credentials.get('url') # url 'wml_credentials'
  
    ibm_api_key = wml_credentials.get('apikey','')  # ibm cloud api_key
    environment = params['environment']  # cloud/on-premise
    token = wml_credentials.get('token','') # wml token
    space_asset_dict = params['space_asset_dict']
    
    inference_project_id = params.get('project_id') # RAG project id



    def validate_answer_against_sources(response_answer, source_documents, similarity_threshold=0.5):
        source_texts = [doc for doc in source_documents]
        answer_embedding = embedding.embed_query(response_answer)
        source_embeddings = embedding.embed_documents(source_texts)
        answer_embedding = np.array(answer_embedding).reshape(1, -1)  # Reshape for cosine_similarity
        source_embeddings = np.array(source_embeddings)
        cosine_scores = cosine_similarity(answer_embedding, source_embeddings)[0]  # Get the first row since answer is 1D
        if any(score > similarity_threshold for score in cosine_scores):
            matching_scores = [score for score in cosine_scores if score > similarity_threshold]
            confidence_score = max(matching_scores)
            return {'isHallucination': False, 'confidence_score': confidence_score}
        else:
            confidence_score = np.mean(cosine_scores)
            return {'isHallucination': True, 'confidence_score': float(confidence_score)}

    def is_hallucination(response_answer, source_documents, threshold_overlap_max=0.3,
                         threshold_overlap_score_concat=0.4):
        stop_words = {
            'a', 'about', 'after', 'all', 'also', 'am', 'an', 'and', 'another', 'any', 'are', 'as', 'at',
            'be', 'because', 'been', 'before', 'being', 'between', 'both', 'but', 'by',
            'came', 'can', 'come', 'could', 'did', 'do', 'each', 'for', 'from', 'get', 'got',
            'has', 'had', 'he', 'have', 'her', 'here', 'him', 'himself', 'his', 'how',
            'i', 'if', 'in', 'into', 'is', 'it', 'like',
            'make', 'many', 'me', 'might', 'more', 'most', 'much', 'must', 'my', 'never', 'now',
            'of', 'on', 'only', 'or', 'other', 'our', 'out', 'over',
            'said', 'same', 'should', 'since', 'some', 'still', 'such',
            'take', 'than', 'that', 'the', 'their', 'them', 'then', 'there', 'these', 'they', 'this',
            'those', 'through', 'to', 'too', 'under', 'up', 'very', 'was', 'way', 'we', 'well', 'were',
            'what', 'where', 'which', 'while', 'who', 'with', 'would', 'you', 'your'
        }

        regex = re.compile(r"\b\w+(?:['-_]\w+)?\b")

        def calculate_textual_overlap(text1, text2):
            # tokenize
            text1_tokens = regex.findall(text1.lower())
            text2_tokens = regex.findall(text2.lower())

            # remove stop words
            text1_tokens = [t for t in text1_tokens if t not in stop_words]
            text2_tokens = [t for t in text2_tokens if t not in stop_words]

            # compute overlap
            if len(text1_tokens) > 0:
                text1_tokens = set(text1_tokens)
                text2_tokens = set(text2_tokens)
                return len(text1_tokens.intersection(text2_tokens)) / len(text1_tokens)
            else:
                return 0

        def overlap_score_concat(generated_text, passages):
            passages_text = ' '.join(passages)
            return calculate_textual_overlap(generated_text, passages_text)

        def overlap_score_max(generated_text, passages):
            return max([calculate_textual_overlap(generated_text, passage_text) for passage_text in passages])

        # Get the LLM response along with source document text
        llm_response_text = response_answer
        source_docs_passages = [doc for doc in source_documents]

        result_overlap_score_max = 0.0
        result_overlap_score_concat = 0.0

        if len(source_docs_passages):

            # Call overlap score calculations
            result_overlap_score_max = overlap_score_max(llm_response_text, source_docs_passages)
            result_overlap_score_concat = overlap_score_concat(llm_response_text, source_docs_passages)

            print('result_overlap_score_max: ' + str(result_overlap_score_max))
            print('result_overlap_score_concat: ' + str(result_overlap_score_concat))

            # Define thresholds (these should be put in a parameter set or template)
            # Need to experiment with what values to use

            response = {'isHallucination': False,
                        'text': "",
                        'maxOverlapScore': round(result_overlap_score_max, 2),
                        'concatOverlapScore': round(result_overlap_score_concat, 2)}

            # Decide if hallucination or not
            if result_overlap_score_max > threshold_overlap_max and result_overlap_score_concat > threshold_overlap_score_concat:

                response['isHallucination'] = False
            else:
                print("Hallucination")
                response['isHallucination'] = True
                response['text'] = "Sorry, I can not generate a valid answer to your question."
        else:
            response = {'isHallucination': True,
                        'text': "Sorry, I can not find an answer to your question in the available documents.",
                        'maxOverlapScore': round(result_overlap_score_max, 2),
                        'concatOverlapScore': round(result_overlap_score_concat, 2)}

        return response

    try:
        client = APIClient(wml_credentials)
        client.set.default_space(params['space_id'])
    except Exception as e:

        print(f"Error initializing WML client: {str(e)}")

        raise

    try:
        helper_function_name = 'rag_helper_functions.py'
        helper_function_path = client.data_assets.download(space_asset_dict[helper_function_name], helper_function_name)
        script_name = os.path.basename(helper_function_path).replace('.py', '')
        rag_helper_functions = __import__(script_name)
        space_parameter_dict = {key: value for key, value in space_asset_dict.items() if "parameter" in key.lower()}

        parameters_list = []
        for i in space_parameter_dict.values():
            parameters_list = parameters_list + client.parameter_sets.get_details(i)['entity']['parameter_set']['parameters']
        parameters = {param['name']: param['value'] for param in parameters_list}

        hallucination_threshold_max_text_overlap = parameters['hallucination_threshold_max_text_overlap']
        hallucination_threshold_concatenated_text_overlap = parameters['hallucination_threshold_concatenated_text_overlap']


    except Exception as e:

        print(f"Error initializing the parameters: {str(e)}")
        raise

    index_name = parameters["vector_store_index_name"]
    model_id = parameters['elastic_search_model_id']
    hybrid_search = True if parameters['milvus_hybrid_search'].lower()=="true" else False
    
    expert_profiles_index = parameters["expert_profiles_index"]
    expert_profile_vector_store = None

    try:
        db_connection = client.connections.get_details(params['connection_id'])['entity']['properties']



        
        if params['connection_type'] == "elasticsearch":
            es_client = rag_helper_functions.create_and_check_elastic_client(db_connection, parameters['elastic_search_model_id'])

            # if expert_profiles_index != '':
            #     expert_profile_es_client = rag_helper_functions.create_and_check_elastic_client(db_connection, parameters['elastic_search_model_id'])

        ############################################# MILVUS #########################################################
        elif params['connection_type'] == "milvus" or params['connection_type'] == "milvuswxd":
            if environment=="cloud":
                embedding = Embeddings(
                model_id=parameters['embedding_model_id'],
                credentials=wml_credentials,
                project_id=inference_project_id,
                verify=True
                )
            elif environment=="on-prem":
                if parameters['wx_ai_inference_space_id']:
                    
                    print("local on prem embeddng models are not found, using models from IBM Cloud API")
                    credentials=Credentials(
                        api_key = parameters['watsonx_ai_api_key'],
                        url =parameters['watsonx_ai_url'])
                    embedding = Embeddings(
                        model_id=parameters['embedding_model_id'],
                        credentials=credentials,
                        space_id=parameters["wx_ai_inference_space_id"],
                        verify=True
                    )
                else:
                    embedding = Embeddings(
                        model_id=parameters['embedding_model_id'],
                        credentials=wml_credentials,
                        project_id=inference_project_id,
                        verify=True
                    )
                
                    
            embedding_dim = len(embedding.embed_query("a"))
            db_connection['database'] = db_connection.get('database', 'default')
            milvus_credentials = rag_helper_functions.connect_to_milvus_database(db_connection, parameters)

            
            
            print("using the model", parameters['embedding_model_id'], "to create dense embeddings")

            dense_index_param = {"metric_type": "L2", "index_type": "IVF_FLAT","params": {"nlist": 1024}}
            
            if hybrid_search:
                sparse_index_param = {"metric_type": "BM25","index_type": "SPARSE_INVERTED_INDEX", "params": {"drop_ratio_build": 0.2}}
                print("using BM25 sparse embeddings.")
                vector_store = Milvus(
                    embedding_function=embedding,
                    builtin_function=BM25BuiltInFunction(output_field_names="sparse"), 
                    index_params=[dense_index_param, sparse_index_param],
                    vector_field=["dense", "sparse"],
                    connection_args=milvus_credentials,
                    primary_field='id',
                    consistency_level="Strong",
                    collection_name=parameters["vector_store_index_name"] 
                )
                print("Milvus Vector Store Created")
            else:
                vector_store = Milvus(
                    embedding_function=embedding,
                    index_params=dense_index_param,
                    connection_args=milvus_credentials,
                    primary_field='id',
                    consistency_level="Strong",
                    collection_name=parameters["vector_store_index_name"] 
                )
                print("Milvus Vector Store Created")
            
            if expert_profiles_index != '':
                expert_profile_vector_store = Milvus(
                    embedding_function=embedding,
                    connection_args=milvus_credentials,
                    index_params=dense_index_param,
                    primary_field='id',
                    collection_name=parameters["expert_profiles_index"]

                )
                print("Milvus Vector Store Created for expert profile")
            
            
            
    ############################################# MILVUS #########################################################

    except Exception as e:

        print(f"Error creating connection and vector store: {str(e)}")

    try:

        # Gen AI via WatsonX.AI

#################### on-prem testing logic ###########################
        if parameters['wx_ai_inference_space_id']:
            print("Running the inference on watsonx.ai cloud from on-prem ..")
            space_credentials = Credentials(api_key=parameters['watsonx_ai_api_key'], url=parameters['watsonx_ai_url'])
            model = ModelInference(
                  deployment_id=params['prompt_deployment_id'],
                  credentials=space_credentials,
                  space_id= parameters['wx_ai_inference_space_id']
                )
#################### on-prem testing logic ###########################
                
        else:
            print("Running the prompt on watsonx.ai..")
            model = ModelInference(
                  deployment_id=params['prompt_deployment_id'],
                  api_client=client
                )

        print("model initialized")

        def custom_body_func(query: str) -> dict:
            print(f"Reading from the template {parameters['elastic_search_template_file']}")
            es_query_template = json.loads(client.data_assets.get_content(space_asset_dict[parameters['elastic_search_template_file']]))
            
            # Replace placeholders with actual values
            es_query_str = json.dumps(es_query_template)  # Convert to string for easier replacement
            es_query_str = es_query_str.replace("{{model_id}}", parameters['elastic_search_model_id'])
            es_query_str = es_query_str.replace("{{model_text}}", query)
            
            # Convert back to dictionary
            es_query_template = json.loads(es_query_str)
            
            #return {"query": es_query_template}
            return es_query_template

        def retrieve_with_scores(query, filter=None):

            # if document merging is active, k_factor estimates the maximum number of documents that can be added due to size reduction by document merge
            k_factor = parameters['ingestion_chunk_size'] / (parameters['ingestion_chunk_size']-parameters['ingestion_chunk_overlap']) if parameters['ingestion_chunk_size'] > parameters['ingestion_chunk_overlap'] else 1.0

            search_kwargs = {
                "k": math.floor(parameters['vectorsearch_top_n_results'] * k_factor),
                "score_threshold": float(parameters['rag_es_min_score']),
                "include_scores": True,
                "verbose": True
            }
            if filter is not None:
                search_kwargs["filter"] = filter


            if params['connection_type'] == "elasticsearch":
                from langchain_elasticsearch import ElasticsearchRetriever

############ Custom function to read from ES query templates ###########


                retriever = ElasticsearchRetriever(
                                es_client=es_client,
                                index_name=parameters["vector_store_index_name"],
                                body_func=custom_body_func,
                                content_field="text",
                                # document_mapper = document_mapper,
                                search_kwargs=search_kwargs
                            )

                
                results = retriever.invoke(query) #vector_store.similarity_search_with_relevance_scores(query, **search_kwargs)
                # docs_with_scores = [{"page_content": doc.page_content, "metadata":doc.metadata['_source']['metadata'], "score": doc.metadata['_score']} for doc in results]
                results_lin = []
                for res in results:
                    if type(res) == list:
                        results_lin.extend(res)
                    else:
                        results_lin.append(res)
                docs_with_scores = [{"page_content": doc.page_content, "metadata":doc.metadata['_source']['metadata'], "score": doc.metadata['_score'] or doc.metadata['_rank']} for doc in results_lin]
                
                
                
            elif params['connection_type'] == "milvus" or params['connection_type'] == "milvuswxd":
                if hybrid_search:
                    results=vector_store.similarity_search_with_score(query,  ranker_type="weighted", ranker_params={"weights": [0.5, 0.5]},**search_kwargs)
                else:
                    results = vector_store.similarity_search_with_score_by_vector(embedding.embed_query(query), **search_kwargs)
                # print(results)
                docs_with_scores = [{"page_content": doc.page_content,"metadata": doc.metadata, "score": score} for doc, score in results]
                # print(docs_with_scores)

            # re-merge subsequent chunks
            # and add more docs if length reduction allows
            docs_with_scores_all = sorted(docs_with_scores, key=lambda x: x['score'], reverse=True)
            last_doc_index = parameters['vectorsearch_top_n_results']
            if last_doc_index > len(docs_with_scores_all):
                last_doc_index = len(docs_with_scores_all)
            docs_with_scores = docs_with_scores_all[:last_doc_index]
            free_capacity = 0
            # loop: merge -> if total length reduction exceeds next docs length: add next docs -> continue
            while True:
                docs_with_scores, length_reduction = rag_helper_functions.merge_documents(docs_with_scores, parameters['document_source_field'] if 'document_source_field' in parameters else '')
                free_capacity = free_capacity + length_reduction
                quit = True
                while last_doc_index < len(docs_with_scores_all) and free_capacity >= len(docs_with_scores_all[last_doc_index]['page_content']):
                    docs_with_scores.append(docs_with_scores_all[last_doc_index])
                    free_capacity = free_capacity - len(docs_with_scores_all[last_doc_index]['page_content'])
                    last_doc_index = last_doc_index + 1
                    quit = False
                if quit:
                    break

            formatted_docs = "".join([f"[Document]\n{d['page_content']}[End]\n\n" for d in docs_with_scores])
            # print(formatted_docs)
            return docs_with_scores, formatted_docs
            
        print("retriever created")

        def check_for_pii_hap(llm_response, pii_flag, hap_flag):
            result_dict = llm_response['response']['results'][0]
            try: 
                if result_dict.get('moderations'):
                
                    if result_dict.get('moderations').get('pii'):
                        
                        print("\nPII detected.. Score : ", result_dict['moderations']['pii'][0]['score'])
                        pii_flag = True
                    elif result_dict.get('moderations').get('hap'):
                        print("\nHAP detected.. Score : ", result_dict['moderations']['hap'][0]['score'])
                        hap_flag = True
                        
                        
                else: 
                    pii_flag = False
            except:
                print("Moderations check failure. Please check the LLM response.")
                pii_flag = False
                hap_flag = False

            print("PII Flag:", pii_flag)
            print("HAP Flag:", hap_flag)
            return pii_flag, hap_flag
        
        def call_runnable_map(inputs):
            query = inputs["query"]
            filter = inputs.get("filter")

            ai_guardrails = True if parameters['ai_guardrails'].strip().lower() == 'true' else False

            guardrails_params = {
                "guardrails": ai_guardrails,
                "guardrails_pii_params": {"threshold": parameters['guardrails_pii_threshold']},
                "guardrails_hap_params": {"threshold": parameters['guardrails_hap_threshold']}
            } if ai_guardrails else {}
            
            llm_runnable = RunnableMap({
                "context": lambda x: x["context"],
                "question": lambda x: x["question"],
                "response": lambda x: model.generate_text(params={"prompt_variables": x}, raw_response=True, **guardrails_params)
            })
            print("llm chain created")
            


        
            docs_with_scores, formatted_context = retrieve_with_scores(query, filter)
    
                
                # Prepare the inputs for the RunnableMap
            runnable_inputs = {
                    "context": formatted_context,
                    "question": query
                }
            llm_response = llm_runnable.invoke(runnable_inputs)
            print(llm_response['response']['results'][0])
            pii_flag=False
            hap_flag=False
            pii_flag, hap_flag=check_for_pii_hap(llm_response, pii_flag, hap_flag)
            return {"answer":llm_response['response']['results'][0]['generated_text'], "context":docs_with_scores, "pii_flag": pii_flag,"hap_flag":hap_flag}



    except Exception as e:

        print(f"Error creating LLM Chain: {str(e)}")

    ################# Logger Class ###########

    # Logger class for updating Elasticsearch log index
    class rag_logger:

        def __init__(self, parameters):
            self.log_client = None
            print("logging", parameters['log_index_name'])
            if not 'log_index_name' in parameters or parameters['log_index_name'] == '':
                self.logger_status = f"logging is deactivated"
            elif not 'log_connection_type' in params or params["log_connection_type"] == '':
                self.logger_status = f"logging is deactivated"
            else:
                self.log_index_name = parameters['log_index_name']
                self.field_names = []
                try:
                    self.log_connection = client.connections.get_details(params['log_connection_id'])['entity']['properties']
                except:
                    self.log_connection = {key[4:]: parameters[key] for key in parameters.keys() if key.startswith('log_')}
                try:
                    
                    self.log_connection_type = params["log_connection_type"]

                    print("Log Connection type is identified as:", self.log_connection_type)
                    if self.log_connection_type == "elasticsearch":
                        self.log_client = rag_helper_functions.create_and_check_elastic_client(self.log_connection, parameters['elastic_search_model_id'])
                        idx_create_status = self.log_client.options(ignore_status=400).indices.create(index=self.log_index_name, body={'mappings': {'properties': {'question': {'type': 'completion'}}}})
                        self.logger_status = f"logger: connection established, log index {self.log_index_name} {'created' if not 'status' in idx_create_status else 'already exists' if idx_create_status['status'] == 400 else 'DEFECTIVE'}"

                    elif self.log_connection_type == "milvus" or self.log_connection_type == "milvuswxd":
                        
                        self.log_connection['database']=self.log_connection.get('database','default')
                        self.log_connection_params = {
                            'database': self.log_connection['database'],
                            'host': self.log_connection['host'],
                            'port': self.log_connection['port'],
                            'user': self.log_connection['username'],
                            'password': self.log_connection['password'],
                            'secure': True
                        }
                        
                        if 'ssl_certificate' in self.log_connection :
                            self.log_ssl_certificate_content = self.log_connection.get('ssl_certificate') if self.log_connection.get('ssl_certificate') else ""
                            self.log_cert_file_path = 'log_conn.cert'
                            with open(self.log_cert_file_path, 'w') as file:
                                file.write(self.log_ssl_certificate_content)
                            self.log_connection_params['server_pem_path'] = self.log_cert_file_path
                        
                        
                        connections.connect(**self.log_connection_params)
                        self.log_client = self.log_connection_type
                        
                        # Create a collection
                        collection_name = parameters['log_index_name']
                        if collection_name not in utility.list_collections():

                            fields = [
                                FieldSchema(name="id", dtype=DataType.VARCHAR, is_primary=True, max_length=65535),
                                FieldSchema(name="question", dtype=DataType.VARCHAR, max_length=65535),
                                FieldSchema(name="response", dtype=DataType.VARCHAR, max_length=65535),
                                FieldSchema(name="source_documents", dtype=DataType.JSON),
                                FieldSchema(name="Hallucination_Detection", dtype=DataType.JSON),
                                FieldSchema(name="log_timestamp", dtype=DataType.VARCHAR, max_length=65535),
                                FieldSchema(name="feedback", dtype=DataType.JSON),
                                FieldSchema(name="expert_details", dtype=DataType.JSON),
                                FieldSchema(name="elapsed_times", dtype=DataType.JSON),
                                FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=embedding_dim)
                                # Ensure the embedding dimension is correct
                            ]

                            self.field_names = [field.name for field in fields]
                            

                            # Create the collection schema
                            schema = CollectionSchema(fields, description="Collection for document embeddings", enable_dynamic_field=True)

                            # Create the index for the embeddings field
                            index_params = {"metric_type": "L2", "index_type": "IVF_FLAT","params": {"nlist": 1024},}
                            self.collection = Collection(name=collection_name, schema=schema, dynamic_field=True)
                            self.collection.create_index(field_name="vector", index_params=index_params)
                            self.logger_status = f"logger: connection established, log index {self.log_index_name}", 'created.'
                        else:
                            self.collection = Collection(name=collection_name)
                            self.field_names = [field.name for field in self.collection.schema.fields]
                            self.logger_status = f"logger: connection established, log index {self.log_index_name}", 'already exists.'

                        self.collection.load()

                        print("Milvus Conection Created")
                except Exception as e:
                    self.logger_status = f"logger: ERROR: {str(e)}"
                    self.log_client = None

                # Personally identifiable information (PII) detection
                self.pii_detection_model = None
                pii_status = 'inactive'
                self.pii_model_syntax = None
                self.pii_model_transformer = None
                self.pii_model_rbr = None
                if watson_nlp_available:
                    gpu_available = False
                    try:
                        hw_spec = os.environ['RUNTIME_HARDWARE_SPEC']
                        if 'num_gpu' in json.loads(hw_spec):
                            gpu_available = True
                    except:
                        pass
                    try:
                        self.pii_model_syntax = watson_nlp.load('syntax_izumo_en_stock')
                        # Bidirectional Long Short-Term Memory (BiLSTM) model for names detection
                        # self.pii_model_bilstm = watson_nlp.load('entity-mentions_bilstm_en_pii')
                        # Transformer model for person name detection
                        self.pii_model_transformer = watson_nlp.load(f"entity-mentions_transformer-workflow_multilingual_slate.153m.distilled{'-cpu' if not gpu_available else ''}")
                        # Rule-Based Reasoner (RBR) for credit / social numbers and email detection
                        self.pii_model_rbr = watson_nlp.load('entity-mentions_rbr_multi_pii')
                        pii_status = 'active'
                    except:
                        pii_status = 'model load failed'
                self.logger_status = f"{self.logger_status}, PII suppression status: {pii_status}"

        def status(self):
            return self.logger_status if self.logger_status else 'unknown'

        def remove_pii(self, text):
            # remove PII from given text
            if self.pii_model_transformer and self.pii_model_rbr:
                for model_run in [lambda text: self.pii_model_transformer.run(text, language_code='en'), lambda text: self.pii_model_rbr.run(text, language_code='en')]:
                    pii_detection = model_run(text)
                    begin = 0
                    parts = []
                    for _pii in pii_detection.mentions:
                        parts.append(text[begin:_pii.span.begin])
                        begin = _pii.span.end
                    parts.append(text[begin:len(text)])
                    text = 'XXX'.join(parts)
            return text

        def generate_hash(self, content):
            return hashlib.sha256(content.encode()).hexdigest()

        def create_log_record(self, question, content, times, feedback):
            log_id = ''
            if self.log_client:
                # compile LOG record (RAG response + question + time meassurements)
                log_record = {**content, 'log_timestamp': datetime.now().isoformat()}
                if question:
                    log_record['question'] = self.remove_pii(question)  # remove PII from user's question ("Hi I am Joe ...")
                if feedback:
                    log_record['feedback'] = feedback
                if content and 'response' in content:
                    log_record['response'] = self.remove_pii(
                        log_record['response'])  # remove PII if genAI refers to user's question ("Hi Joe, ...")
                if times:
                    times.append(('pii_removal', time.perf_counter()))
                    log_record['elapsed_times'] = {times[i][0]: "{:.3f}".format(times[i][1] - times[i - 1][1]) for i in range(1, len(times))}
                res = None
                if self.log_connection_type == 'elasticsearch':
                    res = self.log_client.index(index=self.log_index_name, document=log_record)
                    self.log_client.indices.refresh()
                elif self.log_connection_type == 'milvus' or self.log_connection_type == "milvuswxd":
                    log_id = self.generate_hash(str(datetime.now().timestamp()))
                    log_record.update({'id': log_id, 'vector': embedding.embed_documents([log_record['question']])[0],
                                       'Hallucination_Detection': log_record['Hallucination Detection'],
                                       'feedback': {}, 'expert_details': []})
                    log_record.pop('Hallucination Detection')
                    res = self.collection.insert(data=log_record)
                    self.collection.flush()
                    print(f'inserted data: {res.primary_keys[0]}')
                if res != None:
                    if self.log_connection_type == 'elasticsearch':
                        log_id = res['_id']
                    elif self.log_connection_type == 'milvus' or self.log_connection_type == 'milvuswxd':
                        log_id = res.primary_keys[0]
                    status = 'ok'
                else:
                    status = 'log cannot be written'

            else:
                status = self.logger_status

            return status, log_id
        
        def update_log_record(self, fields):

            if self.log_client:
                if not 'log_id' in fields:
                    status = "log id not provided"
                else:
                    log_id = fields['log_id']
                    fields.pop("log_id")
                    if len(fields.keys()) < 1:
                        # no field values to be updated is provided
                        status = "field to be updated not provided"
                    else:
                        try:
                            if 'expert_details' in fields:
                                update_query = {'expert_details': fields['expert_details']}
                            else:
                                update_query = {'feedback': fields}
                            # update document
                            if self.log_connection_type == 'elasticsearch':
                                idx_upd_status = self.log_client.options(ignore_status=404).update(index=self.log_index_name, id=log_id, doc=update_query)
                                status = "ok" if not 'status' in idx_upd_status else (f"log id {str(log_id)} not found" if idx_upd_status['status'] == 404 else f"update failed, status {str(idx_upd_status['status'])}")
                                
                            elif self.log_connection_type == 'milvus' or self.log_connection_type == "milvuswxd":
                                existing_record = self.collection.query(expr=f"id=='{log_id}'", output_fields=self.field_names)
                                if len(existing_record) >= 1:
                                    existing_record = existing_record.pop()
                                    existing_record.update(update_query)
                                    res = self.collection.upsert(data=existing_record)
                                    status = 'ok'
                                else:
                                    status = f"log id {str(log_id)} not found"
                        except Exception as e:
                            status = f"update of index {self.log_index_name} failed."
            else:
                status = self.logger_status

            return status
       
        #TODO milvus query
        def retrieve_log_record(self, log_id):
            expert_details = []
            question = None
            if self.log_client:
                try:
                    # update document
                    if self.log_connection_type == 'elasticsearch':
                        response = self.log_client.search(index=self.log_index_name, query={'term': {'_id': log_id}})
                        if len(response.body['hits']['hits']) > 0:
                            question = response.body['hits']['hits'][0]['_source']['question']
                            expert_details = response.body['hits']['hits'][0]['_source'].get('expert_details', [])
                                
                    # milvus query
                    elif self.log_connection_type == 'milvus' or self.log_connection_type == "milvuswxd":
                        response = self.collection.query(expr=f"id=='{log_id}'", output_fields=['expert_details', 'question'])
                        if len(response) >= 1:
                            response = response.pop()
                            question = response.get('question')
                            expert_details = response.get('expert_details', [])

                except Exception as e:
                    status = f"Error retrieving log records for the question {question}"
            else:
                status = self.logger_status

            return question, expert_details
        
        def get_completion_options(self, input_autocomplete_fields):            
            autocomplete_dict=[]
            if self.log_client:
                if input_autocomplete_fields['_function'] != "_auto_complete":
                    status = "_autocomplete not provided"
                else:
                    status="ok"
                    input_autocomplete_fields.pop("_function")
                    if len(input_autocomplete_fields.keys()) < 1:
                        # no feedback values
                        status = "input_autocomplete_fields values not provided"
                    else:
                        if not '_question_prefix' in input_autocomplete_fields:
                            status = "_question_prefix not provided"
                        else:
                            if 'limit' not in input_autocomplete_fields:
                                input_autocomplete_fields['limit']=5
                            try:
                                question_prefix=input_autocomplete_fields["_question_prefix"]
                                print(question_prefix)
                                if self.log_connection_type == 'elasticsearch':
                                    body = {'suggest': {'question': {'prefix': question_prefix, 'completion': {'field': 'question', 'skip_duplicates': True, 'size': input_autocomplete_fields['limit']}}}, "_source": [ "response", "question" ]}
                                    res = self.log_client.search(index=self.log_index_name, body=body)
                                    if 'suggest' in res and 'question' in res['suggest'] and len(res['suggest']['question']) > 0 and 'options' in res['suggest']['question'][0]:
                                        options = res['suggest']['question'][0]['options']
                                        autocomplete_dict = [{'question': _option['_source']['question'], 'response': _option['_source']['response']} for _option in options if '_source' in _option and 'question' in _option['_source'] and 'response' in _option['_source']]
                                elif self.log_connection_type == 'milvus' or self.log_connection_type == "milvuswxd":

                                    raw_prefix = input_autocomplete_fields["_question_prefix"].strip()
                                    question_prefix = raw_prefix.replace("'", "\\'").lower()
                                    
                                    user_question_matching_records = self.collection.query(
                                        expr="question != ''",
                                        output_fields=["question"]
                                    )

                                    # Case-insensitive prefix filtering
                                    filtered = [
                                        item for item in user_question_matching_records
                                        if item["question"].strip().lower().startswith(question_prefix)
                                    ]
                        
                                    # Remove duplicates and sort by frequency
                                    filtered = remove_duplicate_questions(filtered)
                                    filtered = sort_questions_by_occurrence(filtered)

                                    autocomplete_dict = filtered[:input_autocomplete_fields['limit']]
                        
                                    if not autocomplete_dict:
                                        return "No log record found!", autocomplete_dict
                        
                                status = "ok"
                            except Exception as e:
                                status = f"Autocomplete failed."+str(e)
            else:
                status = self.logger_status

            return status, autocomplete_dict

    logger = rag_logger({**parameters, **params})
    print(f"log status : {logger.status()}")
    
    def sort_questions_by_occurrence(autocomplete_dict):
        counts = Counter(item["question"].strip().lower() for item in autocomplete_dict)
        return sorted(autocomplete_dict, key=lambda x: counts[x["question"].strip().lower()], reverse=True)

    
    def remove_duplicate_questions(autocomplete_dict):
        seen = set()
        result = []
        for item in autocomplete_dict:
            key = item["question"].strip().lower()
            if key not in seen:
                seen.add(key)
                result.append(item)
        return result
    
    def get_top_experts(fields):
        query = None
        k = 1
        expert_details = []
        status = f"invalid parameters"

        if 'log_id' in fields:
            log_id = fields.pop('log_id')
        else:
            return {"predictions": [{"fields": ["status"], "values": [[status]]}]}
        
        try:
            
            query, expert_details = logger.retrieve_log_record(log_id)
            status = 'expert_details retrieved from log records'

            print('query: ', query)

            if query is None:
                status = f'log id {log_id} not found'
                
            if query is not None and (len(expert_details) == 0 or (len(expert_details) == 1 and expert_details[0]=='')):
                top_k_experts = parameters['top_k_experts']
                es_search_kwargs = {
                    "size": top_k_experts,
                    "min_score": float(parameters['expert_profiles_es_min_score']),
                    # "include_scores": True,
                    # "verbose": True
                }
                
                milvus_search_params = {"metric_type": "L2", "params": {"radius": 1.07}}
                
                if parameters["expert_profiles_index"] == '':#expert_profile_vector_store is None and expert_profile_es_client is None:
                    status = "No expert profile index is provided"
                else:
                    #Retrieve top relevant experts on the query
                    if params['connection_type'] == "elasticsearch":
#                        def custom_body_func(query: str) -> dict:
#                            es_template=json.loads(client.data_assets.get_content(space_asset_dict[parameters['elastic_search_template_file']]))
#                            es_query_str = json.dumps(es_template)  
#                            es_query_str = es_query_str.replace("{{model_id}}", parameters['elastic_search_model_id'])
#                            es_query_str = es_query_str.replace("{{model_text}}", query)
#                            # Convert back to dictionary
#                            es_query_template = json.loads(es_query_str)
#                        
#                            return {"query": es_query_template}

                        es_query = custom_body_func(query)
                        query_temp_args = es_query
                        if 'sub_searches' in es_query:
                            query_temp_args = {'body': es_query}
                            
                        results = es_client.search(index=parameters["expert_profiles_index"], **es_search_kwargs, **query_temp_args)
                        expert_details = [{'expert_id': doc['_source']['metadata']['document_id'], 
                                       'name': doc['_source']['metadata'].get('name', ''), 
                                       'email': doc['_source']['metadata'].get('email', ''),
                                       'phone': doc['_source']['metadata'].get('phone', ''), 
                                       'domain': doc['_source']['metadata'].get('domain', ''), 
                                       'position': doc['_source']['metadata'].get('position', ''), 
                                       'source': doc['_source']['metadata'].get('source', ''), 
                                       'text': doc['_source']['text']} for doc in results['hits']['hits']]
                        
                        # results = expert_profile_vector_store.similarity_search_with_relevance_scores(query, **es_search_kwargs)
                    elif params['connection_type'] == "milvus" or params['connection_type']=="milvuswxd":
                        results = expert_profile_vector_store.similarity_search_with_score_by_vector(embedding.embed_query(query), k=top_k_experts, param=milvus_search_params)
                        expert_details = [{'expert_id': doc[0].dict()['metadata']['document_id'], 
                                       'name': doc[0].dict()['metadata'].get('name', ''), 
                                       'email': doc[0].dict()['metadata'].get('email', ''), 
                                       'phone': doc[0].dict()['metadata'].get('phone', ''), 
                                       'domain': doc[0].dict()['metadata'].get('domain', ''), 
                                       'position': doc[0].dict()['metadata'].get('position', ''), 
                                       'source': doc[0].dict()['metadata'].get('source', ''), 
                                       'text': doc[0].dict()['page_content']} for doc in results]

                    #print('results: ',results)
                    

                    if len(expert_details) > 0:
                        #Update the log record with expert_details
                        expert_update_log = {'log_id': log_id, 'expert_details': expert_details}
                        status = logger.update_log_record(expert_update_log)
                        status = "log updated with expert_details" if status == 'ok' else status
                    else:
                        status = "No relevant expert profile is available"
        except Exception as e:
            status = f'Error recommending top experts: {e}'
            
        expert_response = {
            "predictions": [
                {
                    "fields": [
                        "recommended_top_experts", "status"
                    ],
                    "values": [[expert_details, status]]
                }]}
        
        return expert_response
    

    def score(payload):

        logging = True
        question = None
        query_filter = None
        status = f"invalid parameters"
        fields = {payload['input_data'][0]['fields'][i]: payload['input_data'][0]['values'][0][i] for i in range(min(len(payload['input_data'][0]['fields']), len(payload['input_data'][0]['values'][0])))}

        if '_log' in fields:
            logging = fields.pop('_log')

        if 'question' in fields:
            question = fields.pop('question')

        if '_content' in fields:
            print('calling insert')
            (status, log_id) = logger.create_log_record(None, fields['_content'], None, None) if logger else (f"log not available", '')
            return {"predictions": [{"fields": ["status", "log_id"], "values": [[status, log_id]]}]}
        
        if '_function' in fields and fields['_function'] == 'recommend_top_experts':
            expert_response = get_top_experts(fields)
            return expert_response

        if '_function' in fields and fields['_function'] == '_auto_complete':
            # initialize time measurement list
            times = [('start', time.perf_counter())]
            completion_status, completion_options = logger.get_completion_options(fields) if logger else f"feedback log not available"
            times.append(('get_completion_options', time.perf_counter()))
            return {"predictions": [ { "fields": ["status", "options", "elapsed_times"], "values": [[completion_status, completion_options, { times[i][0]: "{:.3f}".format(times[i][1] - times[i-1][1]) for i in range(1, len(times)) }]] } ]}

        
        if 'log_id' in fields:
            status = logger.update_log_record(fields) if logger else f"log not available"

        if 'query_filter' in fields:
            query_filter = fields.pop('query_filter')

        else:

            if not question and len(fields.keys()) == 1:
                question = payload['input_data'][0]['values'][0][0]
                fields = {}

        if not question:
            return {"predictions": [{"fields": ["status"], "values": [[status]]}]}

        # initialize time measurement list
        times = [('start', time.perf_counter())]

        try:

            if query_filter:

                inputs = {
                    "query": question,
                    "filter": [{"match": query_filter}]
                }
            else:
                inputs = {"query": question}



            
            # Get the response using the RunnableMap
            llm_response = call_runnable_map(inputs)

            times.append(('llm_chain', time.perf_counter()))
            scoring_response_txt = {}
            scoring_response_txt['response'] = llm_response['answer']

            hallucination_response_text = "Sorry, I cannot find an answer to your question in the available documents."

            if llm_response['context'] and llm_response['pii_flag'] == False and llm_response['hap_flag'] == False:
                llm_response_documents = [i['page_content'] for i in llm_response['context']]
                

                if parameters['default_hallucination_technique'] == "sentence_transformer":
                    hallucination_response = validate_answer_against_sources(llm_response['answer'], llm_response_documents, parameters["similarity_threshold_for_sentence_transformer_hallucination"])
                    hallucination_dict = {"Technique": "sentence_transformer", "confidence_score": hallucination_response['confidence_score']}
                elif parameters['default_hallucination_technique'] == "word_overlap":
                    hallucination_response = is_hallucination(llm_response['answer'], llm_response_documents, hallucination_threshold_max_text_overlap, hallucination_threshold_concatenated_text_overlap)
                    hallucination_dict = {"Technique": "Word Overlap", 
                                          "text_max_overlap_score": hallucination_response['maxOverlapScore'],
                                          "concatenated_text_overlap_score": hallucination_response['concatOverlapScore']}

                else:
                    hallucination_dict = {"Technique": "None"}
                    hallucination_response = {"isHallucination": False}
                if hallucination_response['isHallucination']:
                    scoring_response_txt['response'] = hallucination_response_text
                
                scoring_response_txt['source_documents'] = [doc for doc in llm_response['context']] #[{**response['document'].dict(), 'score': response['score']} for response in llm_response['context']]  # 

                scoring_response_txt['Hallucination Detection'] = hallucination_dict
            elif llm_response['pii_flag']:
                scoring_response_txt['response'] = "There was Personally Identifiable Information detected in the input context."
                scoring_response_txt['source_documents'] = [doc for doc in llm_response['context']]
                hallucination_dict = {"Technique": "None"}
                hallucination_response = {"isHallucination": False}
                scoring_response_txt['Hallucination Detection'] = hallucination_dict
            elif llm_response['hap_flag']:
                scoring_response_txt['response'] = "There was Harmful, Abusive, or Profane (HAP) detected in the input context."
                scoring_response_txt['source_documents'] = [doc for doc in llm_response['context']]
                hallucination_dict = {"Technique": "None"}
                hallucination_response = {"isHallucination": False}
                scoring_response_txt['Hallucination Detection'] = hallucination_dict


            else:
                scoring_response_txt['response'] = hallucination_response_text
                scoring_response_txt['Hallucination Detection'] = { "Technique": parameters['default_hallucination_technique']}

            times.append(('hallucination_detection', time.perf_counter()))



        except Exception as e:

            print(f"Error while getting LLM response: {str(e)}")
            raise


        # provide set of urls without duplicates
        try:
            document_title_field = 'metadata.title'.split('.')
            document_url_field = 'metadata.document_url'.split('.')
            urls = sorted([{'title': reduce(lambda a,i: a[i], document_title_field, _doc) if not document_title_field == '' else '', \
                            'url': reduce(lambda a,i: a[i], document_url_field, _doc) if not document_url_field == '' else '', \
                            'score': _doc['score'] if 'score' in _doc else 0.0} for _doc in scoring_response_txt['source_documents']], key=lambda x: x['title']+"#"+x['url'])
            scoring_response_txt['source_documents_references'] = \
                sorted([_url for _i, _url in enumerate(urls) if not (_url['title'] == '' and _url['url'] == '') and (_i == 0 or not _url['title'] == urls[_i-1]['title'] or not _url['url'] == urls[_i-1]['url'])], \
                       key=lambda x: x['score'], reverse=True)
        except KeyError:
            scoring_response_txt['source_documents_references'] = []

            
        # post log record and return log id to caller
        if logging:
            scoring_response_txt['log_status'], scoring_response_txt['log_id'] = logger.create_log_record(question, scoring_response_txt, times, fields)
            
        
        # Retrieve top recomended experts and post it to log record.
        if logging and parameters['expert_profiles_index'] != '':
            fields.update({'log_id': scoring_response_txt['log_id']})
            expert_response = get_top_experts(fields)

        
        scoring_response = {
            "predictions": [
                {
                    "fields": [
                        "llm_prediction"
                    ],
                    "values": [[scoring_response_txt]]
                }]}

        return scoring_response
    
    
    return score

Uncomment the below cells to test the scoring pipeline function locally in scope of the notebook. 

**Note:** To run the function locally you need to also uncomment all the subprocess calls to install the dependent libraries. Similarly, ensure that they are commented out before the function is deployed in the Deployment space to avoid library conflicts. (In very rare cases a kernel restart may be required.)

In [ ]:
#rag_scoring_pipeline_function()({"input_data": [{"fields": ["question"],"values": [["How do I create a connection to a database from a watsonx.ai project?"]]}]})

We can filter the source documents title with a value `SQL optimization (SPSS Modeler)` and LLM will generate the response from only those source documents using the below cell.

In [ ]:
#rag_scoring_pipeline_function()({"input_data": [{"fields": ["question", "query_filter"],"values": [["how to perform sql optimization?",{"metadata.title.keyword": {"query": "SQL optimization (SPSS Modeler)"}}]]}]})

Optionally, we can further retrieve top recommended experts by providing the log_id for the question.

In [ ]:
# rag_scoring_pipeline_function()({"input_data": [{"fields": ['_function', "log_id"],"values": [['recommend_top_experts', '']]}]})

We can also auto-complete questions from the log index. By default this is limited to 5 questions. We can update this value to any number bewtween 1-10

In [ ]:
#rag_scoring_pipeline_function()({"input_data": [{ "fields": ['_function', "_question_prefix", "limit"],"values": [['_auto_complete', "how to", 5]] }]})

<a id="DeployScoringFunction"></a>
### Deployment of function to Space

The code first defines metadata for the `rag_scoring_pipeline_function`, including its name, description, and software specification, and stores the function in the deployment space.

In [ ]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'rag_function_watsonx_ai_with_'+connection_type,
    client.repository.FunctionMetaNames.DESCRIPTION: 'QnA with RAG using ' + connection_type,
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sw_spec_id
}

watsonx_function_details = client.repository.store_function(meta_props=meta_data, function=rag_scoring_pipeline_function)
watsonx_function_uid = client.repository.get_function_id(watsonx_function_details)

After storing the function in deployment space, the below code performs the following tasks:

- It checks if the desired `deployment_serving_name` is available for use with the function.
- If the serving name is available, a new deployment is created for the scoring function with the specified serving name and metadata (name, description, hardware spec).
- If the serving name already exists, it retrieves the details of the existing deployment, extracts its ID, and updates the deployment's assets with the new scoring function.
- The deployment ID is then retrieved after either creating or updating the deployment for future use.

In [ ]:
try:
    if(client.deployments.is_serving_name_available(parameters['deployment_serving_name'])):
        print(f"Serving name {parameters['deployment_serving_name']} available")
        
        meta_props = {
           client.deployments.ConfigurationMetaNames.NAME: "rag_scoring_function_with_"+connection_type,
           client.deployments.ConfigurationMetaNames.DESCRIPTION: "QnA with RAG using" + connection_type,
           client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S'},  
           client.deployments.ConfigurationMetaNames.SERVING_NAME:  parameters['deployment_serving_name']
        }

        print(f"Creating a new deployment with the serving name {parameters['deployment_serving_name']}")
        
        watsonx_deployment_details = client.deployments.create(watsonx_function_uid, meta_props=meta_props)
        
        watsonx_deployment_id = client.deployments.get_id(watsonx_deployment_details)

    else:
        print(f"Serving name '{parameters['deployment_serving_name']}' already exists")
        existing_serving_name = client.deployments.get_details(serving_name=parameters['deployment_serving_name'])

        if not existing_serving_name['resources']:
            print("Serving name not accessible from the deployment space")
            raise RuntimeError("Error accessing the serving name from the deployment space. Please update the deployment serving name in parameter set")
        
        print("Fetching the deployment details from the serving name..")
        
        watsonx_deployment_id = existing_serving_name['resources'][0]['metadata']['id']
        metadata = {client.deployments.ConfigurationMetaNames.ASSET: { "id": watsonx_function_details['metadata']['id'] }}
        print("Updating the assets of the deployment..")

        if client.deployments.get_details(watsonx_deployment_id)['entity']['status']['state'] == "ready":
            watsonx_deployment_details = client.deployments.update(watsonx_deployment_id, changes=metadata)
        

        timeout = 900  
        start_time = time.time()
        
        while True:
            status = client.deployments.get_details(watsonx_deployment_id)['entity']['status']['state']
            print(f"Current status: {status}")
            
            if status == 'ready':
                print("The assets of the scoring pipeline function deployment are successfully updated!")
                break
            
            elapsed_time = time.time() - start_time
            if elapsed_time > timeout:
                print("The update process timed out after waiting for 15 minutes.")
                break
            print("Update in progress... Please wait.")
            time.sleep(20)  

except Exception as e:
    raise RuntimeError(f"An error occurred: {e}")


<a id="scoring"></a>
### Test the deployed scoring function

Test the deployed scoring function by passing in a question.

In [ ]:
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["question"],
        "values": [["how to perform decision optimization?"]]
    }]
}
wx_result = client.deployments.score(watsonx_deployment_id, payload)
wx_result

<a id="expert-recommendation"></a>
### Test the expert recommendation function

Test the deployed scoring function to recommend top experts for the question asked by passing the log_id. This cell will only work and give results if you have executed the **Ingest Expert Profile data to vector DB** notebook. 

In [ ]:
try:
    log_id = wx_result['predictions'][0]['values'][0][0]['log_id']
except NameError: 
    log_id = ''

payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ['_function', "log_id"],
        "values": [['recommend_top_experts', log_id]]
       
    }]
}
wx_log_update_result = client.deployments.score(watsonx_deployment_id, payload)
wx_log_update_result

<a id="auto complete support"></a>
### Test Auto complete

Test the deployed scoring function to auto-complete the given question prefix with existing user questions logged into log index. Update `<_add_question_prefix>` & limit values in the `values` parameter below to test. <br>
**Note**: This functionality support requires the log index schema for the question field to have auto completion support in case of ES. 
Please update it with a new log index parameter. Existing/Old indexes will not directly support this functionality. Lastly, also ensure your log index has sufficient records to match the given question prefix

In [ ]:
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ['_function', "_question_prefix", "limit"],
        "values": [['_auto_complete', "<_add_question_prefix>", 5]]
       
    }]
}
wx_log_update_result = client.deployments.score(watsonx_deployment_id, payload)
wx_log_update_result

<a id="feedback-logging"></a>
### Feedback Logging

Updating feedback in log record. In case the answer returned by the LLM is relevant to your question provide the values below as postive along with a comment. Otherwise, provide the values below as negative. \
This feedback will be saved if an Elasticsearch index or Milvus collection is provided for feedback logging in the parameter set. 
In case of elasticsearch, you can view the indexed feedback logs via the elasticsearch **kibana** user interface under the **Index Management** section. \
Find the document for the log record that corresponds to the conversation above. It contains, the question and answers, the documents found and the user's feedback.

In [ ]:
try:
    log_id = wx_result['predictions'][0]['values'][0][0]['log_id']
except NameError: 
    log_id = ''
    
payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        "fields": ["log_id", "value", "comment"],
        "values": [[log_id, "positive", "Nice log record!"]]
       
    }]
}
wx_log_update_result = client.deployments.score(watsonx_deployment_id, payload)
wx_log_update_result

<a id="updateParameters"></a>
### Update parameter set in the project & deployment space

Update the advanced parameter set in both project & space with the deployment id of the function. 

In [ ]:
paramset_name = "RAG_advanced_parameter_set"
parameter_to_be_updated = {"name":"wml_rag_deployment_id","value":watsonx_deployment_id}
rag_helper_functions.update_parameter_set(client,paramset_name,parameter_to_be_updated)

client.set.default_project(project_id=project_id)
if rag_helper_functions.update_parameter_set(client,paramset_name,parameter_to_be_updated) == True:
    print("Parameter set in the project and deployment space has been updated with the deployment id of the function above.")
else:
    print("Parameter set update failed.")
client.set.default_space(space_uid)

**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2024, 2025. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**